# Recommendation System

In this notebook, I explored three kinds of recommendation - popularity, content-based, and colaborative filter recommendations. The final result should be that anytime our customers input one movie's name, the system can provide them the other most possible movies they want to watch.

## Ingest

In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

import surprise
from surprise import Dataset
from surprise import Reader
from surprise import SVDpp
from surprise import model_selection
from surprise.model_selection import cross_validate, GridSearchCV

In [2]:
df_rating = pd.read_csv('rating.csv')
df_anime = pd.read_csv('anime.csv')
print("Rating's shape: {}".format(df_rating.shape))
print("Anime's shape: {}".format(df_anime.shape))

Rating's shape: (7813737, 3)
Anime's shape: (12294, 7)


In [3]:
print(df_rating.head())
print(df_rating.rating.unique())

   user_id  anime_id  rating
0        1        20      -1
1        1        24      -1
2        1        79      -1
3        1       226      -1
4        1       241      -1
[-1 10  8  6  9  7  3  5  4  1  2]


-1 means people didn't give the movies they have seen ratings.

In [4]:
df_anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


## The Simplest Recommendation System
For the most direct recommendation, we can just find the most popular movies and it is probably that most of the people will like them. However, just using average rating of the moives is not a very good way because it cannot take popularity and audience's amount into consideration. Therefore, I used another way to give every movie a score. 

Weighted Rating(WR) = v*R/(v+m) + m*C/(v+m)

v is the number of votes for the movie;

m is the minimum votes required to be listed

R is the average rating of the movie

C is the average rating of all movies

### Data Processing

In [5]:
# Remove null values (-1)
df_rating = df_rating[df_rating.rating != -1]
df_rating.shape

(6337241, 3)

In [6]:
df_anime_rec1 = df_anime.drop(['genre', 'type', 'episodes', 'members'], axis=1)
df_rating_rec1 = df_rating.drop(['rating', 'user_id'], axis=1)
df_anime_rating_total = df_anime_rec1.merge(df_rating_rec1, how='inner', on='anime_id')
df_anime_rating_total.head()

,anime_id,name,rating
0,32281,Kimi no Na wa.,9.37
1,32281,Kimi no Na wa.,9.37
2,32281,Kimi no Na wa.,9.37
3,32281,Kimi no Na wa.,9.37
4,32281,Kimi no Na wa.,9.37


In [7]:
df_anime_rating_total = df_anime_rating_total.dropna()
df_anime_rating_number = df_anime_rating_total.groupby(['anime_id'], as_index=False)['rating'].count()
df_anime_rating_number = df_anime_rating_number.rename(columns={'rating': 'rating number'})
df_anime_rating_number.head()

,anime_id,rating number
0,1,13449
1,5,5790
2,6,9385
3,7,2169
4,8,308


In [8]:
df_anime_rating_total = df_anime_rating_total.merge(df_anime_rating_number, on='anime_id', how='inner')
df_anime_rating_total.head()

,anime_id,name,rating,rating number
0,32281,Kimi no Na wa.,9.37,1961
1,32281,Kimi no Na wa.,9.37,1961
2,32281,Kimi no Na wa.,9.37,1961
3,32281,Kimi no Na wa.,9.37,1961
4,32281,Kimi no Na wa.,9.37,1961


In [9]:
df_anime_rating = df_anime_rating_total.drop_duplicates(keep='first')
df_anime_rating = df_anime_rating.reset_index()
df_anime_rating = df_anime_rating.drop(['index'], axis=1)
df_anime_rating.head()

,anime_id,name,rating,rating number
0,32281,Kimi no Na wa.,9.37,1961
1,5114,Fullmetal Alchemist: Brotherhood,9.26,21494
2,28977,Gintama°,9.25,1188
3,9253,Steins;Gate,9.17,17151
4,9969,Gintama&#039;,9.16,3115


In [10]:
warnings.filterwarnings('ignore')

C = df_anime_rating['rating'].mean()
m = df_anime_rating['rating number'].quantile(0.9)
df_anime_rating_recommend = df_anime_rating[df_anime_rating['rating number'] >= m]
df_anime_rating_recommend['scoring'] = df_anime_rating_recommend['rating number']*df_anime_rating_recommend['rating']/(df_anime_rating_recommend['rating number']+m) + m*C/(df_anime_rating_recommend['rating number']+m)
df_anime_rating_recommend.head()

,anime_id,name,rating,rating number,scoring
0,32281,Kimi no Na wa.,9.37,1961,8.082243
1,5114,Fullmetal Alchemist: Brotherhood,9.26,21494,9.065249
3,9253,Steins;Gate,9.17,17151,8.938426
4,9969,Gintama&#039;,9.16,3115,8.255402
6,11061,Hunter x Hunter (2011),9.13,7477,8.661400


### Provide Recommendations

In [11]:
df_anime_rating_recommend = df_anime_rating_recommend.sort_values('scoring', ascending=False)
df_anime_rating_recommend.head(10)

,anime_id,name,rating,rating number,scoring
1,5114,Fullmetal Alchemist: Brotherhood,9.26,21494,9.065249
3,9253,Steins;Gate,9.17,17151,8.938426
10,4181,Clannad: After Story,9.06,15518,8.817249
13,2904,Code Geass: Hangyaku no Lelouch R2,8.98,21124,8.802825
15,199,Sen to Chihiro no Kamikakushi,8.93,19481,8.743065
19,1575,Code Geass: Hangyaku no Lelouch,8.83,24126,8.683245
6,11061,Hunter x Hunter (2011),9.13,7477,8.661400
39,1535,Death Note,8.71,34226,8.610159
29,2001,Tengen Toppa Gurren Lagann,8.78,16955,8.581330
22,1,Cowboy Bebop,8.82,13449,8.570855


In [54]:
# Show detailed information about those most popular movies
df_anime_popular = pd.DataFrame({'anime_id':[], 'name':[], 'genre':[], 'type':[], 'episodes':[], 'rating':[], 'members':[]})

df_anime_popular_name = df_anime_rating_recommend.head(10)['name']
for name in df_anime_popular_name:
    df_anime_popular = df_anime_popular.append(df_anime[df_anime['name'] == name])
        
df_anime_popular = df_anime_popular.reset_index()
df_anime_popular = df_anime_popular.drop(['index'], axis=1)
df_anime_popular

,anime_id,name,genre,type,episodes,rating,members
0,5114.0,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665.0
1,9253.0,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572.0
2,4181.0,Clannad: After Story,"Drama, Fantasy, Romance, Slice of Life, Supern...",TV,24,9.06,456749.0
3,2904.0,Code Geass: Hangyaku no Lelouch R2,"Action, Drama, Mecha, Military, Sci-Fi, Super ...",TV,25,8.98,572888.0
4,199.0,Sen to Chihiro no Kamikakushi,"Adventure, Drama, Supernatural",Movie,1,8.93,466254.0
5,1575.0,Code Geass: Hangyaku no Lelouch,"Action, Mecha, Military, School, Sci-Fi, Super...",TV,25,8.83,715151.0
6,11061.0,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855.0
7,1535.0,Death Note,"Mystery, Police, Psychological, Supernatural, ...",TV,37,8.71,1013917.0
8,2001.0,Tengen Toppa Gurren Lagann,"Action, Adventure, Comedy, Mecha, Sci-Fi",TV,27,8.78,562962.0
9,1.0,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824.0


## Content-Based Recommendation System
For this kind of recommender, it is based on movies' genres. Anytime one customer inputs a movie's name, the system will provide him or her with 10 possible movies that are similar to the one customer chooses. I will apply TF-IDF to 'genre' and you can find more introduction about TF-IDF in my Github (https://github.com/Zhenyu0521/Text-Analysis/blob/master/NLP%20for%20Yelp%20Reviews/NLP_for_Yelp_Reviews.ipynb)

### Data Processing

In [13]:
df_anime.isna().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [14]:
df_anime_cbrs = df_anime.dropna(axis=0)
df_anime_cbrs = df_anime_cbrs.reset_index()
df_anime_cbrs.isna().sum()

index       0
anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [15]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_anime_cbrs['genre'])

# Calculate cosine similarities
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

### Provide Recommendations

In [56]:
indices = pd.Series(df_anime_cbrs.index, index=df_anime_cbrs['name']).drop_duplicates()

def get_recommendations_cb(title, cosine_sim=cosine_sim):
    index = indices[title]
    sim_scores = list(enumerate(cosine_sim[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df_anime['name'].iloc[movie_indices]

In [57]:
get_recommendations_cb('Fullmetal Alchemist: Brotherhood')

200                               Fullmetal Alchemist
1558    Fullmetal Alchemist: The Sacred Star of Milos
402         Fullmetal Alchemist: Brotherhood Specials
879               Tales of Vesperia: The First Strike
4262            Tetsujin 28-gou: Hakuchuu no Zangetsu
1967                 Fullmetal Alchemist: Reflections
101                        Magi: The Kingdom of Magic
268                      Magi: The Labyrinth of Magic
290                       Magi: Sinbad no Bouken (TV)
461                            Magi: Sinbad no Bouken
Name: name, dtype: object

## Collaborative Filtering Recommendation System

### Exploratory Data Analysis

In [18]:
df_rating.head()

,user_id,anime_id,rating
47,1,8074,10
81,1,11617,10
83,1,11757,10
101,1,15451,10
153,2,11771,10


In [19]:
data = df_rating['rating'].value_counts().sort_index(ascending=False)
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / df_rating.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
               )

layout = dict(title = 'Distribution Of {} moive-ratings'.format(df_rating.shape[0]),
              xaxis = dict(title = 'Rating'),
              yaxis = dict(title = 'Count'))

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

Since the rating data has more than seven million lines, I just selected fifty thousands to show the function of colaborative filtering system.

In [31]:
df = df_rating.iloc[:50000,].reset_index()
df = df.drop(['index'], axis=1)
df.head()

,user_id,anime_id,rating
0,1,8074,10
1,1,11617,10
2,1,11757,10
3,1,15451,10
4,2,11771,10


### Build up Recommendation Model

In [32]:
df['rating'].unique()

array([10,  8,  6,  9,  7,  3,  5,  4,  1,  2])

In [52]:
recmodel = SVDpp()
reader = Reader(rating_scale=(1,10))
df_rating_rec = Dataset.load_from_df(df, reader)
recmodel.fit(df_rating_rec.build_full_trainset()) 
cross_validate(recmodel, df_rating_rec, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2433  1.2393  1.2216  1.2350  1.2365  1.2351  0.0073  
MAE (testset)     0.9409  0.9408  0.9270  0.9345  0.9295  0.9346  0.0057  
Fit time          79.34   82.53   79.97   81.07   78.76   80.33   1.34    
Test time         1.78    1.33    1.42    1.37    1.52    1.48    0.16    


{'test_rmse': array([1.24331433, 1.23932615, 1.22160432, 1.23498246, 1.23652093]),
 'test_mae': array([0.94093514, 0.9408233 , 0.92704923, 0.93453507, 0.9295186 ]),
 'fit_time': (79.34301090240479,
  82.53376007080078,
  79.96587181091309,
  81.06647229194641,
  78.75729584693909),
 'test_time': (1.7804927825927734,
  1.3315300941467285,
  1.4163670539855957,
  1.3749818801879883,
  1.5157160758972168)}

In [37]:
anime_id = df['anime_id'].unique()
# just take user1 for example
anime_id1 = df.loc[df['user_id'] == 1, 'anime_id']
anime_id_to_pred = np.setdiff1d(anime_id, anime_id1)

In [43]:
testset = [[1, anime_id, 10] for anime_id in anime_id_to_pred]
user_id1_pred = recmodel.test(testset)
df_pred = pd.DataFrame(user_id1_pred)
df_pred.head()

,uid,iid,r_ui,est,details
0,1,1,10,9.344287,{'was_impossible': False}
1,1,5,10,9.185070,{'was_impossible': False}
2,1,6,10,8.901061,{'was_impossible': False}
3,1,7,10,8.368037,{'was_impossible': False}
4,1,8,10,8.422988,{'was_impossible': False}


In [44]:
df_pred = df_pred.rename(columns={'uid': 'user_id', 'iid': 'anime_id', 'est': 'predicted rating'})
df_pred = df_pred.drop(['r_ui', 'details'], axis=1)
df_pred.head()

,user_id,anime_id,predicted rating
0,1,1,9.344287
1,1,5,9.185070
2,1,6,8.901061
3,1,7,8.368037
4,1,8,8.422988


In [47]:
df_pred = df_pred.sort_values('predicted rating', ascending=False)
df_pred_anime_id = df_pred.head(10)['anime_id']

df_recommendation = pd.DataFrame({'anime_id':[], 'name':[], 'genre':[], 'type':[], 'episodes':[], 'rating':[], 'members':[]})

for anime_id in df_pred_anime_id:
    df_recommendation = df_recommendation.append(df_anime[df_anime['anime_id'] == anime_id])
        
df_recommendation = df_recommendation.reset_index()
df_recommendation = df_recommendation.drop(['index'], axis=1)
df_recommendation['anime_id'] = df_recommendation['anime_id'].astype('int')
df_recommendation

,anime_id,name,genre,type,episodes,rating,members
0,19815,No Game No Life,"Adventure, Comedy, Ecchi, Fantasy, Game, Super...",TV,12,8.47,602291.0
1,12355,Ookami Kodomo no Ame to Yuki,"Fantasy, Slice of Life",Movie,1,8.84,226193.0
2,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572.0
4,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855.0
5,16894,Kuroko no Basket 2nd Season,"Comedy, School, Shounen, Sports",TV,25,8.58,243325.0
6,6114,Rainbow: Nisha Rokubou no Shichinin,"Drama, Historical, Seinen, Thriller",TV,26,8.64,139474.0
7,13759,Sakurasou no Pet na Kanojo,"Comedy, Drama, Romance, School, Slice of Life",TV,24,8.40,380375.0
8,245,Great Teacher Onizuka,"Comedy, Drama, School, Shounen, Slice of Life",TV,43,8.77,268487.0
9,263,Hajime no Ippo,"Comedy, Drama, Shounen, Sports",TV,75,8.83,157670.0


### Provide Recommendations

In [59]:
def get_recommendations_cf(user_id, num_recommendations):
    """Provide recommendations for specific user with the number they want to show.
    """
    anime_id = df['anime_id'].unique()
    anime_id_user = df.loc[df['user_id'] == user_id, 'anime_id']
    anime_id_to_pred = np.setdiff1d(anime_id, anime_id_user)
    testset = [[user_id, anime_id, 10] for anime_id in anime_id_to_pred]
    user_id_pred = recmodel.test(testset)
    df_pred = pd.DataFrame(user_id_pred)
    
    df_pred = df_pred.sort_values('est', ascending=False)
    df_pred_anime_id = df_pred.head(num_recommendations)['iid']

    df_recommendation = pd.DataFrame({'anime_id':[], 'name':[], 'genre':[], 'type':[], 'episodes':[], 'rating':[], 'members':[]})

    for anime_id in df_pred_anime_id:
        df_recommendation = df_recommendation.append(df_anime[df_anime['anime_id'] == anime_id])
        
    df_recommendation = df_recommendation.reset_index()
    df_recommendation = df_recommendation.drop(['index'], axis=1)
    df_recommendation['anime_id'] = df_recommendation['anime_id'].astype('int')
    return df_recommendation

In [60]:
get_recommendations_cf(1, 15)

,anime_id,name,genre,type,episodes,rating,members
0,16498,Shingeki no Kyojin,"Action, Drama, Fantasy, Shounen, Super Power",TV,25,8.54,896229.0
1,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572.0
2,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855.0
3,2904,Code Geass: Hangyaku no Lelouch R2,"Action, Drama, Mecha, Military, Sci-Fi, Super ...",TV,25,8.98,572888.0
4,245,Great Teacher Onizuka,"Comedy, Drama, School, Shounen, Slice of Life",TV,43,8.77,268487.0
5,12355,Ookami Kodomo no Ame to Yuki,"Fantasy, Slice of Life",Movie,1,8.84,226193.0
6,199,Sen to Chihiro no Kamikakushi,"Adventure, Drama, Supernatural",Movie,1,8.93,466254.0
7,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0
8,19815,No Game No Life,"Adventure, Comedy, Ecchi, Fantasy, Game, Super...",TV,12,8.47,602291.0
9,9989,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada...,"Drama, Slice of Life, Supernatural",TV,11,8.62,463835.0


## Conclusion

### Popularity Recommendation System

In [55]:
df_anime_popular

,anime_id,name,genre,type,episodes,rating,members
0,5114.0,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665.0
1,9253.0,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572.0
2,4181.0,Clannad: After Story,"Drama, Fantasy, Romance, Slice of Life, Supern...",TV,24,9.06,456749.0
3,2904.0,Code Geass: Hangyaku no Lelouch R2,"Action, Drama, Mecha, Military, Sci-Fi, Super ...",TV,25,8.98,572888.0
4,199.0,Sen to Chihiro no Kamikakushi,"Adventure, Drama, Supernatural",Movie,1,8.93,466254.0
5,1575.0,Code Geass: Hangyaku no Lelouch,"Action, Mecha, Military, School, Sci-Fi, Super...",TV,25,8.83,715151.0
6,11061.0,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855.0
7,1535.0,Death Note,"Mystery, Police, Psychological, Supernatural, ...",TV,37,8.71,1013917.0
8,2001.0,Tengen Toppa Gurren Lagann,"Action, Adventure, Comedy, Mecha, Sci-Fi",TV,27,8.78,562962.0
9,1.0,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824.0


### Content-Based Recommendation System

In [58]:
get_recommendations_cb('Fullmetal Alchemist: Brotherhood')

200                               Fullmetal Alchemist
1558    Fullmetal Alchemist: The Sacred Star of Milos
402         Fullmetal Alchemist: Brotherhood Specials
879               Tales of Vesperia: The First Strike
4262            Tetsujin 28-gou: Hakuchuu no Zangetsu
1967                 Fullmetal Alchemist: Reflections
101                        Magi: The Kingdom of Magic
268                      Magi: The Labyrinth of Magic
290                       Magi: Sinbad no Bouken (TV)
461                            Magi: Sinbad no Bouken
Name: name, dtype: object

### Collaborative Filtering Recommendation System - User to User

In [61]:
get_recommendations_cf(2, 5)

,anime_id,name,genre,type,episodes,rating,members
0,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855.0
1,16498,Shingeki no Kyojin,"Action, Drama, Fantasy, Shounen, Super Power",TV,25,8.54,896229.0
2,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572.0
3,12355,Ookami Kodomo no Ame to Yuki,"Fantasy, Slice of Life",Movie,1,8.84,226193.0
4,1535,Death Note,"Mystery, Police, Psychological, Supernatural, ...",TV,37,8.71,1013917.0


## Improvement
Seems that the best way to provide recommendations is to combine these three types of systems and give comprehensive suggestions. It needs me to spend much more time to find a proper method to combine them and get better model.